In [9]:
import gzip
import pandas as pd
import numpy as np
import json

In [30]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

def getDF(path,columns=None,users=None):
    i = 0
    df = {}
    for d in parse(path):
        if columns != None:
            d = {i:d[i] for i in columns}
        if users!=None:
            if d['reviewerID'] in users:
                df[i] = d
                i += 1
        else:
            df[i] = d
            i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [11]:
df = getDF('./data/Clothing_Shoes_and_Jewelry.json.gz',{'reviewerID','asin'})

In [12]:
df.head()

,reviewerID,asin
0,A2IC3NZN488KWK,0871167042
1,A3OT9BYASFGU2X,0871167042
2,A28GK1G2KDXHRP,0871167042
3,A3NFXFEKW8OK0E,0871167042
4,A3I6G5TKBVJEK9,0871167042


In [13]:
df_filtered = df.drop_duplicates()

In [17]:
df_filtered.shape, df.shape

((31663536, 2), (32292099, 2))

In [18]:
users_consumption= df_filtered.groupby('reviewerID').count()

In [24]:
top_users = users_consumption.loc[users_consumption['asin'] >=5]

In [29]:
top_users=top_users.sample(100000)

In [33]:
top_users_set = set(top_users.index.values)

In [34]:
final_df = getDF('./data/Clothing_Shoes_and_Jewelry.json.gz',users=top_users_set)

In [39]:
final_df.groupby('reviewerID')['asin'].count().value_counts()[:20]

5     22917
6     17153
7     12448
8      8865
9      6798
10     5325
11     4212
12     3296
13     2595
14     2136
15     1806
16     1476
17     1335
18     1106
19      964
20      832
21      691
22      576
23      512
24      478
Name: asin, dtype: int64

In [41]:
final_df.to_parquet('./data/amazon_clothing.parquet')